In [1]:
from faster_whisper import WhisperModel

C:\Users\Hp\Desktop\Cours\repos_githubs\voiceOver_app\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_size = "large-v3"
model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe("../audio.mp3", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

Detected language 'en' with probability 0.860063


In [4]:
segments = list(segments) 
segments

[Segment(id=1, seek=0, start=0.72, end=11.36, text=' okay this is supposed to be a test audio test let us see how my model to transcribe my speech', tokens=[50401, 1392, 341, 307, 3442, 281, 312, 257, 1500, 6278, 1500, 718, 505, 536, 577, 452, 2316, 281, 1145, 8056, 452, 6218, 50933], avg_logprob=-0.17530460962477853, compression_ratio=1.3333333333333333, no_speech_prob=0.04572983831167221, words=None, temperature=0.0),
 Segment(id=2, seek=0, start=11.36, end=15.92, text=' into text will perform thank you very much', tokens=[50933, 666, 2487, 486, 2042, 1309, 291, 588, 709, 51161], avg_logprob=-0.17530460962477853, compression_ratio=1.3333333333333333, no_speech_prob=0.04572983831167221, words=None, temperature=0.0)]

In [8]:
doc = """"""
for segment in segments:
    doc = doc + "[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text) + '\n'

print(doc)

[0.72s -> 11.36s]  okay this is supposed to be a test audio test let us see how my model to transcribe my speech
[11.36s -> 15.92s]  into text will perform thank you very much



In [23]:
from moviepy import VideoFileClip, AudioFileClip

from pathlib import Path

import uuid
import json
from pathlib import Path
from typing import List, Dict, Tuple, Optional


In [13]:

DEFAULT_SR = 16000
TARGET_AUDIO_SR = 22050  # XTTS works nicely here
MAX_REF_SECONDS = 60      # reference voice audio total for cloning
MIN_REF_SECONDS = 15


In [35]:
def extract_audio(video_path: str, out_wav: str, sr: int = DEFAULT_SR) -> str:
    clip = VideoFileClip(video_path)
    audio = clip.audio
    if audio is None:
        raise RuntimeError("No audio track found in video.")
    audio.write_audiofile(out_wav, fps=sr, codec="pcm_s16le", 
                          #verbose=True, 
                          logger=None
                         )
    clip.close()
    return out_wav

In [24]:
def download_youtube(url: str, tmpdir: str) -> Optional[str]:
    try:
        import yt_dlp
        outtmpl = str(Path(tmpdir) / f"yt_{uuid.uuid4().hex}.%(ext)s")
        ydl_opts = {
            "outtmpl": outtmpl,
            "format": "mp4/bestaudio/best",
            "noplaylist": True,
            "quiet": True,
            "merge_output_format": "mp4",
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            filepath = ydl.prepare_filename(info)
            # Ensure .mp4 extension after merge
            if not filepath.endswith(".mp4"):
                filepath = Path(filepath).with_suffix(".mp4")
            return str(filepath)
    except Exception as e:
        raise RuntimeError(f"YouTube download failed: {e}")
        return None


In [28]:
dl_vid = download_youtube(url='https://www.youtube.com/watch?v=PphHVMtWB6E', 
                 tmpdir=r'C:\Users\Hp\Desktop\Cours\repos_githubs\voiceOver_app')

In [29]:
wav_path = "audio3.wav"

In [32]:
extract_audio(video_path=dl_vid, out_wav=wav_path)

'audio3.wav'

In [33]:
model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe("audio3.wav", beam_size=5)

In [34]:
doc = """"""
for segment in segments:
    doc = doc + "[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text) + '\n'

print(doc)

[0.00s -> 23.52s]  C.S.M.
[30.00s -> 59.98s]  C.S.M.
[59.98s -> 65.66s]  A special tribute to the General Secretary, Apostle Alexander Naniakumilabi.
[72.24s -> 79.44s]  As you may be aware, this council meeting marks the last Apostle Alexander Naniakumilabi
[79.44s -> 84.12s]  will be attending in his official capacity as General Secretary.
[84.68s -> 89.46s]  It marks the end of his second five-year tenure as a General Secretary.
[89.46s -> 89.52s]  Thank you.
[89.90s -> 89.96s]  Thank you.
[89.96s -> 89.98s]  Thank you.
[89.98s -> 96.48s]  And you agree with me that he has served selflessly and faithfully in the last ten years.
[98.76s -> 100.14s]  My brother, please come.
[106.38s -> 108.84s]  Shall we put our hands together for the Lord?
[108.84s -> 109.84s]  Thank you.
[109.84s -> 129.84s]  Thank you, Jesus.
[129.84s -> 130.86s]  Yeah, he's my good brother.
[132.48s -> 135.32s]  He has been of great help to me.
[136.86s -> 139.32s]  See, people like us, we do not know anything.
[

In [1]:
import os
import io
import math
import tempfile
import uuid
import json
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import streamlit as st
import numpy as np

# Video/Audio
from moviepy import VideoFileClip, AudioFileClip
from pydub import AudioSegment

# Transcription
from faster_whisper import WhisperModel

# Diarization
from pyannote.audio import Pipeline as PyannotePipeline

# Translation
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline as hf_pipeline

# TTS (voice cloning)
from TTS.api import TTS

C:\Users\Hp\Desktop\Cours\repos_githubs\voiceOver_app\venv\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
C:\Users\Hp\Desktop\Cours\repos_githubs\voiceOver_app\venv\lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
C:\Users\Hp\Desktop\Cours\repos_githubs\voiceOver_app\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Hp\Desktop\Cours\repos_githubs\voiceOver_app\venv\lib\sit

In [2]:
def load_translator(model_name: str = "facebook/nllb-200-distilled-600M"):
    tok = AutoTokenizer.from_pretrained(model_name)
    mod = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    translator = hf_pipeline("translation", model=mod, tokenizer=tok, device=0 if torch_cuda_available() else -1)
    return translator

In [4]:
#st.info("Loading translator (NLLB-200 distilled 600M)...")
translator = load_translator()

#Map Whisper language (like 'en') to NLLB code; if unknown, assume eng
#A simple mapping for common languages:
WHISPER_TO_NLLB = {
    "en": "eng_Latn", "fr": "fra_Latn", "es": "spa_Latn", "de": "deu_Latn",
    "it": "ita_Latn", "pt": "por_Latn", "ru": "rus_Cyrl", "ar": "arb_Arab",
    "hi": "hin_Deva", "sw": "swh_Latn", "tr": "tur_Latn", "pl": "pol_Latn",
    "nl": "nld_Latn", "ja": "jpn_Jpan", "ko": "kor_Hang", "zh": "zho_Hans",
}
src_lang_nllb = WHISPER_TO_NLLB.get(transcript.get("language", "en"), "eng_Latn")


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/models/facebook/nllb-200-distilled-600M/tree/main/additional_chat_templates?recursive=False&expand=False (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000253E8924E80>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 3b021a14-e209-4ce2-96b2-7b4e88cf726a)')

In [ ]:
translations = {}
for lang_name in target_langs:
    tgt_code = LANG_CHOICES[lang_name]
    st.info(f"Translating to {lang_name} ...")
    try:
   translations[lang_name] = VO.translate_text(translator, main_speaker_text, src_lang_nllb, tgt_code)
  except Exception as e:
      st.error(f"Translation to {lang_name} failed: {e}")

In [5]:
import tempfile

In [6]:
tempfile.mkdtemp(prefix="dubbing_")

'C:\\Users\\Hp\\AppData\\Local\\Temp\\dubbing_z9grtw9k'

In [18]:
tempfile.mkdtemp(prefix="dubbing_", dir=os.getcwd())

'C:\\Users\\Hp\\Desktop\\Cours\\repos_githubs\\voiceOver_app\\Sandbox\\dubbing_5vaf6tl6'

In [20]:
os.getcwd()

AttributeError: module 'os' has no attribute 'getcwdr'